<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:


      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 11:11:29
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -96.97 K (-0.69%)
Current PnL: -19.12 L (-12.86%)
CY Booked + Current PnL: -7.68 L (-5.17%)
-------------------
Total profit:  1.52 L
Total loss:  -20.64 L
-------------------
Total Booked + Current PnL: 19.30 L (15.86%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 85.06 L (60.55%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.79%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.76,69.0,H-MC,3.52,199680.0,35010.0,8327.0,1.68,21.26,4.17,26.31,89.0,4.20,1.42,30.94,X40N,NTT,AC
77,TTKPRESTIG,770.00,96.58,40.0,M-SC,3.11,84698.0,-16079.0,16169.0,-1.56,-15.96,19.09,0.09,245.0,-0.99,0.60,11.27,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.22,110452.0,9948.0,19163.0,-1.20,9.90,17.35,28.96,79.0,0.52,0.79,47.23,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,46.0,H-LC,4.66,215888.0,5522.0,23251.0,-0.48,2.62,10.77,13.68,37.0,0.24,1.54,19.58,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3797.30,58.0,M-SC,5.31,121552.0,-25816.0,77635.0,2.61,-17.52,63.87,35.16,236.0,-0.33,0.87,24.15,XY24,NTT,AUTO
14,BLUESTARCO,2080.0,15.76,69.0,H-MC,3.52,199680.0,35010.0,8327.0,1.68,21.26,4.17,26.31,89.0,4.20,1.42,30.94,X40N,NTT,AC
66,SIS,528.0,2077.72,37.0,H-SC,4.84,87886.0,-23146.0,46755.0,1.13,-20.85,53.20,21.26,156.0,-0.50,0.63,18.74,OX40N,NTT,MISC
67,SONACOMS,1006.0,-40.52,39.0,M-SC,11.70,74104.0,-27055.0,101945.0,0.68,-26.74,137.57,74.03,202.0,-0.27,0.53,4.49,AR,BTT,AUTO
58,REPCOHOME,880.0,-59.76,50.0,H-SC,5.59,229742.0,-56763.0,314103.0,0.61,-19.81,136.72,89.82,134.0,-0.18,1.64,21.55,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-64.66,53.0,H-MC,3.74,251261.0,-50490.0,211914.0,-2.95,-16.73,84.34,53.50,116.0,-0.24,1.79,11.60,XY24,NTT,FMCG
64,SHALBY,327.0,1217.38,73.0,M-SC,8.02,179203.0,-1940.0,47417.0,-2.65,-1.07,26.46,25.10,235.0,-0.04,1.28,43.61,XY24,NTT,HEALTHCARE
10,BANDHANBNK,400.0,-14.68,39.0,H-SC,4.69,213516.0,-65043.0,322495.0,-2.36,-23.35,151.04,92.42,151.0,-0.20,1.52,23.40,XY24,NTT,BANKS
6,ASIANTILES,137.0,6927.78,58.0,L-SC,5.23,77091.0,-16719.0,93334.0,-1.96,-17.82,121.07,81.67,269.0,-0.18,0.55,49.29,XR,NTT,CERAMICS
45,KANSAINER,340.0,-67.61,54.0,H-SC,0.87,223371.0,-46296.0,82625.0,-1.90,-17.17,36.99,13.47,138.0,-0.56,1.59,12.98,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-20.54,47.0,H-MC,2.06,136764.0,976.0,41781.0,-0.71,0.72,30.55,31.48,107.0,0.02,0.97,12.92,X40,ATH,INSURANCE
23,DABUR,735.00,-7.80,38.0,H-MC,6.88,249654.0,1586.0,106827.0,-1.40,0.64,42.79,43.70,102.0,0.01,1.78,15.18,XY24,BTT,FMCG
52,PGHH,17973.08,-30.28,53.0,H-MC,3.79,201975.0,1155.0,67621.0,-0.51,0.58,33.48,34.25,80.0,0.02,1.44,5.80,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,9.26,120231.0,-22248.0,41768.0,-0.54,-15.61,34.74,13.70,163.0,-0.53,0.86,41.91,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,96.58,40.0,M-SC,3.11,84698.0,-16079.0,16169.0,-1.56,-15.96,19.09,0.09,245.0,-0.99,0.60,11.27,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.61,54.0,H-SC,0.87,223371.0,-46296.0,82625.0,-1.90,-17.17,36.99,13.47,138.0,-0.56,1.59,12.98,XY24,NTT,PAINTS
18,CERA,9475.0,-21.24,40.0,H-SC,2.09,143532.0,-32371.0,74393.0,-0.98,-18.40,51.83,23.89,149.0,-0.44,1.02,23.96,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.53,42.0,H-MC,7.11,104880.0,-26955.0,69179.0,-1.55,-20.45,65.96,32.03,98.0,-0.39,0.75,17.47,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.76,69.0,H-MC,3.52,199680.0,35010.0,8327.0,1.68,21.26,4.17,26.31,89.0,4.2,1.42,30.94,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,48.36,69.0,M-MC,8.91,234140.0,9178.0,158466.0,0.35,4.08,67.68,74.53,192.0,0.06,1.67,37.65,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.76,69.0,H-MC,3.52,199680.0,35010.0,8327.0,1.68,21.26,4.17,26.31,89.0,4.20,1.42,30.94,X40N,NTT,AC
17,CAMS,5211.76,-2.06,56.0,H-SC,2.64,111294.0,9290.0,34635.0,-0.98,9.11,31.12,43.06,122.0,0.27,0.79,27.78,X40N,ATH,MISC
84,VOLTAS,1530.00,-2.62,44.0,H-MC,2.82,204225.0,12483.0,25283.0,-1.07,6.51,12.38,19.69,99.0,0.49,1.45,13.75,XY25,NTT,AC
23,DABUR,735.00,-7.80,38.0,H-MC,6.88,249654.0,1586.0,106827.0,-1.40,0.64,42.79,43.70,102.0,0.01,1.78,15.18,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,59.0,H-MC,9.22,110452.0,9948.0,19163.0,-1.20,9.90,17.35,28.96,79.0,0.52,0.79,47.23,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.30,47.0,M-SC,1.13,99292.0,7794.0,70954.0,-0.40,8.52,71.46,86.07,223.0,0.11,0.71,44.19,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,65.0,H-MC,6.96,184791.0,4983.0,107437.0,-0.32,2.77,58.14,62.52,88.0,0.05,1.32,37.80,XR,NTT,BANKS
37,INDIAMART,4979.75,-53.52,32.0,H-SC,10.24,129585.0,6249.0,134341.0,-0.18,5.07,103.67,113.99,119.0,0.05,0.92,26.97,AR,ATH,MISC
86,WIPRO,420.00,-11.98,48.0,M-LC,5.68,154727.0,3782.0,105679.0,-0.31,2.51,68.30,72.51,53.0,0.04,1.10,8.68,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.24,36.0,H-LC,7.15,294437.0,-21405.0,134057.0,-0.53,-6.78,45.53,35.67,5.0,-0.16,2.10,6.14,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-14.89,38.0,H-LC,7.65,214984.0,-36784.0,89992.0,-1.60,-14.61,41.86,21.13,27.0,-0.41,1.53,15.64,X40,ATH,PAINTS
30,HCLTECH,1943.91,-2.37,38.0,H-LC,9.94,223223.0,-18701.0,81967.0,-1.26,-7.73,36.72,26.15,8.0,-0.23,1.59,5.48,X40,ATH,IT
19,CIPLA,1795.00,-21.84,40.0,H-LC,5.29,208459.0,3959.0,37460.0,-1.34,1.94,17.97,20.25,10.0,0.11,1.48,10.84,X40N,BTT,PHARMA
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,47.0,H-LC,1.45,234990.0,-11161.0,90377.0,-0.72,-4.53,38.46,32.18,16.0,-0.12,1.67,29.87,X200,ATH,IT
42,ITC,452.0,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
65,SIEMENS,4671.5,0.81,55.0,H-LC,3.33,162150.0,-23945.0,71427.0,-0.02,-12.87,44.05,25.51,15.0,-0.34,1.15,19.49,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-9.62,50.0,H-LC,4.61,278062.0,12636.0,44156.0,-0.52,4.76,15.88,21.40,11.0,0.29,1.98,12.92,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,46.0,H-LC,4.66,215888.0,5522.0,23251.0,-0.48,2.62,10.77,13.68,37.0,0.24,1.54,19.58,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287546.0,-58410.0,133278.0,-0.48,-16.88,46.35,21.64,1.0,-0.44,2.05,1.86,X40,ATH,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.80,52.0,H-LC,7.54,228571.0,-5573.0,54194.0,-0.82,-2.38,23.71,20.77,86.0,-0.10,1.63,4.37,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-2.37,38.0,H-LC,9.94,223223.0,-18701.0,81967.0,-1.26,-7.73,36.72,26.15,8.0,-0.23,1.59,5.48,X40,ATH,IT
83,VBL,671.64,-18.24,36.0,H-LC,7.15,294437.0,-21405.0,134057.0,-0.53,-6.78,45.53,35.67,5.0,-0.16,2.10,6.14,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.81,55.0,H-LC,3.33,162150.0,-23945.0,71427.0,-0.02,-12.87,44.05,25.51,15.0,-0.34,1.15,19.49,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
19,CIPLA,1795.00,-21.84,40.0,H-LC,5.29,208459.0,3959.0,37460.0,-1.34,1.94,17.97,20.25,10.0,0.11,1.48,10.84,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.89,38.0,H-LC,7.65,214984.0,-36784.0,89992.0,-1.60,-14.61,41.86,21.13,27.0,-0.41,1.53,15.64,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,46.0,H-LC,4.66,215888.0,5522.0,23251.0,-0.48,2.62,10.77,13.68,37.0,0.24,1.54,19.58,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.72,43.0,H-LC,12.16,287546.0,-58410.0,133278.0,-0.48,-16.88,46.35,21.64,1.0,-0.44,2.05,1.86,X40,ATH,IT
40,INFY,2275.00,-18.16,48.0,H-LC,8.12,318542.0,5654.0,166024.0,-0.27,1.81,52.12,54.87,3.0,0.03,2.27,8.58,X40,BTT,IT
42,ITC,452.00,-39.19,40.0,H-LC,2.06,197043.0,-3095.0,23980.0,-0.98,-1.55,12.17,10.44,4.0,-0.13,1.40,3.93,X40,NTT,FMCG
83,VBL,671.64,-18.24,36.0,H-LC,7.15,294437.0,-21405.0,134057.0,-0.53,-6.78,45.53,35.67,5.0,-0.16,2.10,6.14,X40N,ATH,FMCG
1,ABB,7934.00,-39.09,53.0,H-LC,9.04,254390.0,-7229.0,126432.0,-1.17,-2.76,49.70,45.57,7.0,-0.06,1.81,8.25,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6927.78,58.0,L-SC,5.23,77091.0,-16719.0,93334.0,-1.96,-17.82,121.07,81.67,269.0,-0.18,0.55,49.29,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-45.67,47.0,L-SC,42.21,81734.0,-31815.0,71869.0,-0.91,-28.02,87.93,35.27,268.0,-0.44,0.58,100.05,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-172.73,60.0,M-SC,4.47,83286.0,-18924.0,112311.0,-1.66,-18.51,134.85,91.37,208.0,-0.17,0.59,35.20,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-6.52,41.0,H-SC,12.04,86946.0,-14017.0,108552.0,-0.30,-13.88,124.85,93.63,148.0,-0.13,0.62,29.50,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,49.0,H-SC,12.67,93915.0,-4065.0,25667.0,-0.57,-4.15,27.33,22.05,152.0,-0.16,0.67,36.51,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1217.38,73.0,M-SC,8.02,179203.0,-1940.0,47417.0,-2.65,-1.07,26.46,25.10,235.0,-0.04,1.28,43.61,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3797.30,58.0,M-SC,5.31,121552.0,-25816.0,77635.0,2.61,-17.52,63.87,35.16,236.0,-0.33,0.87,24.15,XY24,NTT,AUTO
71,TANLA,1963.11,-16.22,69.0,H-SC,14.35,224344.0,-46417.0,376359.0,-0.27,-17.14,167.76,121.86,133.0,-0.12,1.60,75.50,AR,ATH,IT
61,SAMMAANCAP,326.00,-172.73,60.0,M-SC,4.47,83286.0,-18924.0,112311.0,-1.66,-18.51,134.85,91.37,208.0,-0.17,0.59,35.20,XY25,NTT,FINANCE
81,VAIBHAVGBL,521.00,60.33,61.0,H-SC,6.04,139065.0,-43710.0,154779.0,-1.65,-23.91,111.30,60.77,125.0,-0.28,0.99,24.56,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.30
1,25,43.68
2,50,74.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.03
LC    32.39
MC    23.59
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.36
X40      14.71
X40N     11.43
XR       11.26
XY25     10.07
AR        8.41
OX40N     7.82
X200      1.67
MH        1.64
X5K       1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.38
H-LC    25.92
H-MC    20.64
M-SC    13.16
M-LC     5.43
M-MC     2.65
L-SC     1.49
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.96,-4.78,39.37
IT,12.95,-15.69,78.78
FINANCE,10.06,-16.09,66.74
MISC,7.06,-15.63,80.59
BANKS,6.07,-17.24,78.92
PAINTS,5.62,-16.90,34.32
ELECTRICAL,5.49,-6.68,44.10
INSURANCE,3.71,-5.97,45.59
AC,3.57,3.50,12.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3117988.0,22
XR,1330688.0,14
AR,1157101.0,9
X40,782261.0,10
X40N,599853.0,9
OX40N,548824.0,10
XY25,426198.0,7
SR,249327.0,2
X5K,119725.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3673703.0,29
M-SC,1399666.0,17
H-MC,1209268.0,15
H-LC,1180305.0,15
M-LC,385983.0,4
M-MC,306158.0,2
L-SC,251459.0,3
L-MC,60680.0,1
L-LC,38655.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1229056.0      6
M-SC       XY24       799837.0      7
H-SC       AR         794403.0      5
           XR         775951.0      7
H-MC       XY24       557335.0      4
H-LC       X40        495241.0      5
M-MC       XY24       306158.0      2
H-SC       X40N       301569.0      4
           OX40N      259025.0      4
           SR         249327.0      2
H-LC       X40N       225711.0      3
H-MC       X40        214443.0      4
H-LC       AR         197859.0      2
H-MC       XY25       169138.0      2
L-SC       XR         165203.0      2
M-SC       AR         164839.0      2
M-LC       XY24       160579.0      2
M-SC       OX40N      134364.0      4
M-LC       X5K        119725.0      1
M-SC       XR         115738.0      2
           XY25       112311.0      1
H-MC       XR         107437.0      1
H-LC       XY25       106094.0      3
M-LC       XR         105679.0      1
H-LC       X200        90377.0      1
L-SC       OX40N       86256.0      1
M-SC       X40         72577.0      1
H-MC       X40N        72573.0      2
           OX40N       69179.0      1
H-LC       XY24        65023.0      1
H-SC       MH          64372.0      1
L-MC       XR          60680.0      1
L-LC       XY25        38655.0      1
H-MC       MH          19163.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
